In [1]:
import pandas
import math
from sklearn import linear_model
from sklearn.metrics import *


def saveDFToExcel(dataFrame,fileName,sheetName):
  regressionDF = pandas.DataFrame(dataFrame)
  regressionDF.to_excel(fileName + '.xlsx', sheet_name=sheetName,index=False)

df = pandas.read_excel("Factors (with market & industries).xlsx", "Factors")
rollingSize = 6
offsetMonths = 36 
industryCount = 50

for i in range(1,industryCount+1):
  rollingAverage = []
  for monthIndex in range(0,offsetMonths):
    rollingAverage.append(None)
  for monthIndex in range(offsetMonths,len(df)-1):
    rollingWindow = df["Industry"+str(i)][monthIndex-rollingSize+1:monthIndex+1]
    avg = sum(rollingWindow) / rollingSize
    rollingAverage.append(avg)
  rollingAverage.append(None)
  df["rollingAverage"+str(i)] = rollingAverage
  
saveDFToExcel(df,"IndustryMomentumRollingAverages","RollingAverages")


In [2]:
acceptedIndustriesList = []
for monthIndex in range(offsetMonths, len(df)):
  allRollings = []
  for i in range(1,industryCount+1):
    allRollings.append(df["rollingAverage"+str(i)][monthIndex])
  allRollings.sort()
  lastAcceptable = allRollings[int(industryCount * -0.1)]
  acceptedIndustries = []
  for i in range(1,industryCount+1):
    if df["rollingAverage"+str(i)][monthIndex]>= lastAcceptable:
      acceptedIndustries.append(i)
  acceptedIndustriesList.append(acceptedIndustries)

print(acceptedIndustriesList[:20])

[[12, 23, 27, 31, 50], [3, 26, 27, 31, 50], [26, 27, 31, 35, 44], [6, 26, 27, 35, 45], [4, 26, 27, 39, 45], [5, 6, 21, 26, 45, 49], [15, 18, 21, 40, 45, 49], [6, 9, 10, 15, 18, 21], [6, 9, 10, 15, 18, 30], [6, 9, 12, 15, 18, 30], [9, 15, 18, 25, 30], [7, 10, 12, 19, 23, 25], [6, 7, 12, 25, 34], [7, 10, 12, 23, 34], [7, 10, 20, 28, 34], [1, 10, 20, 28, 34], [1, 20, 34, 38, 41], [1, 26, 28, 34, 41], [1, 4, 20, 26, 34], [1, 4, 26, 38, 40]]


In [3]:
strategyYield = 1
monthlyYieldHistory = {"month":[],"yield":[],"strategyYield":[]}
for monthIndex in range(offsetMonths, len(df)-1):
  weight = 1/ int(industryCount * 0.1)
  monthYield = 0
  for acceptedIndustry in acceptedIndustriesList[monthIndex-offsetMonths]:
      monthYield += weight* df["Industry"+str(acceptedIndustry)][monthIndex+1]
  strategyYield *= 1+monthYield
  monthlyYieldHistory["month"].append(df["month"][monthIndex+1])
  monthlyYieldHistory["yield"].append(monthYield)
  monthlyYieldHistory["strategyYield"].append(strategyYield)

print(strategyYield)

saveDFToExcel(monthlyYieldHistory,"IndustryMomentumStrategy","Monthly yields")

1.1946160684626892
